# 🚀 MLflow Tracking for the Sentiment Classifier with Logistic Regression

This code demonstrates how to use MLflow for tracking and logging our well-known sentiment prediction model for the women’s clothing shop dataset, which we used throughout the classes over the past few weeks.

In [1]:
import os
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("Logistic-Regression-Experiment")

print("Connected to:", mlflow.get_tracking_uri())

2026/02/23 20:10:45 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/23 20:10:45 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/23 20:10:45 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/23 20:10:45 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/23 20:10:45 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/23 20:10:45 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/23 20:10:45 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/23 20:10:45 INFO alembic.runtime.migration: Will assume non-transactional DDL.


Connected to: sqlite:///mlflow.db


In [3]:
file_path = 'womens_clothing_ecommerce_reviews.csv'
df = pd.read_csv(file_path)
print("Successfully loaded the dataset.")
print("Dataset preview:")
print(df.head())

Successfully loaded the dataset.
Dataset preview:
                                         Review Text  sentiment
0  Absolutely wonderful - silky and sexy and comf...          1
1  Love this dress!  it's sooo pretty.  i happene...          1
2  I love, love, love this jumpsuit. it's fun, fl...          1
3  This shirt is very flattering to all due to th...          1
4  I love tracy reese dresses, but this one is no...         -1


In [4]:
# --- Step 2: Split Data into Training and Testing Sets ---
# It's crucial to test our model on data it has never seen before.
# We'll use 80% of the data for training and 20% for testing.
X = df['Review Text']
y = df['sentiment']

# 'stratify=y' ensures that the proportion of positive and negative reviews is the same in both your training set and your testing set.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"\nData split into {len(X_train)} training samples and {len(X_test)} testing samples.")


Data split into 15854 training samples and 3964 testing samples.


In [5]:
# --- Step 3: Feature Engineering with Bag-of-Words ---
# Here, we convert the text reviews into numerical feature vectors.
# Each feature is a count of how many times a word appears in a review.
print("\nConverting text to numerical features using Bag-of-Words...")

# Initialize the vectorizer. `stop_words='english'` removes common
# English words like 'the', 'a', 'is', which don't carry much sentiment.
vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizer on the TRAINING data and transform it into a matrix
X_train_bow = vectorizer.fit_transform(X_train)

print(f"X_train_bow Shape:\n {X_train_bow.shape}")

print(f"Unique words (features): {vectorizer.get_feature_names_out()}")

# Output is a sparse matrix representation, where most entries are zero and only non-zero values are stored
print(f"X_train_bow:\n {X_train_bow}")

# ONLY transform the TESTING data using the already-fitted vectorizer
X_test_bow = vectorizer.transform(X_test)

print("✅ Text successfully converted to feature vectors.")


Converting text to numerical features using Bag-of-Words...
X_train_bow Shape:
 (15854, 11897)
Unique words (features): ['00' '000' '00p' ... 'zooming' 'zuma' 'ã¼ber']
X_train_bow:
 <Compressed Sparse Row sparse matrix of dtype 'int64'
	with 373991 stored elements and shape (15854, 11897)>
  Coords	Values
  (0, 10214)	1
  (0, 2396)	1
  (0, 4805)	1
  (0, 4210)	1
  (0, 3835)	1
  (0, 9289)	1
  (0, 3680)	1
  (0, 4669)	1
  (0, 6980)	1
  (0, 10773)	1
  (0, 7248)	1
  (0, 7409)	1
  (1, 9289)	1
  (1, 2866)	1
  (1, 3454)	1
  (1, 4206)	1
  (1, 1781)	1
  (1, 6221)	1
  (1, 6711)	1
  (1, 8861)	1
  (1, 9671)	1
  (1, 9424)	1
  (1, 11)	1
  (2, 7409)	1
  (2, 3454)	3
  :	:
  (15853, 4805)	1
  (15853, 7596)	1
  (15853, 6257)	1
  (15853, 2390)	1
  (15853, 11495)	1
  (15853, 1602)	1
  (15853, 8053)	1
  (15853, 4742)	1
  (15853, 8467)	1
  (15853, 7829)	1
  (15853, 9466)	1
  (15853, 10594)	1
  (15853, 11408)	1
  (15853, 11414)	1
  (15853, 8195)	1
  (15853, 584)	1
  (15853, 11438)	1
  (15853, 5070)	1
  (15853

In [ ]:
with mlflow.start_run(run_name="logistic_regression_baseline"):

    
    # --- Step 4: Train a Logistic Regression Model ---
    print("\nTraining Logistic Regression model...")
    model = LogisticRegression(max_iter=1000, solver='saga', random_state=42)  # max_iter increased to ensure convergence
    model.fit(X_train_bow, y_train)
    print("Model training complete.")
    
    # --- Step 5: Evaluate the Model ---
    print("\nEvaluating model on test set...")
    y_pred = model.predict(X_test_bow)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy: {accuracy:.4f}")
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")

    # Params (include requested fields)
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_param("solver", "saga")
    mlflow.log_param("max_iter", 1000)
    mlflow.log_param("random_state", 42)
    
    # Log the accuracy metric to MLflow
    mlflow.log_metric("test_accuracy", accuracy)
    mlflow.log_metric("test_precision", precision)
    mlflow.log_metric("test_recall", recall)
    mlflow.log_metric("test_f1", f1)

    mlflow.sklearn.log_model(model, name="logistic_model")
    print("Logistic run logged. Accuracy:", accuracy, "Precision:", precision, "Recall:", recall, "F1:", f1)


Training Logistic Regression model...
Model training complete.

Evaluating model on test set...
Test Accuracy: 0.9299
Test Precision: 0.8483
Test Recall: 0.7988
Test F1 Score: 0.8209


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/mlflow/models/model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Logistic run logged. Accuracy: 0.9298688193743693 Precision: 0.8482978549505802 Recall: 0.7988212252003917 F1: 0.8208967837899686
